In [2]:
# # Step 0: Setup and Test
# from IPython.display import HTML
# from matplotlib import animation
# import matplotlib.pyplot as plt
# from helpers import *

# dataPath = "../sensorium_data/"
# mouseId = "dynamic29156-11-10-Video-8744edeac3b4d1ce16b680916b5267ce"

# vid = np.load(dataPath + mouseId + "/data/videos/0.npy")
# vid = reshape_video_hwd_to_dhw(vid)

# %matplotlib inline
# # an equal aspect (square), and turn the axes off
# fig, ax = plt.subplots()
# ax.set_aspect("equal")
# ax.set_axis_off()

# # Images are generated and stored in a list to animate later
# images_vid = []
# images_as_array = []
# for image in vid:
#     # Scatter plot each point using a dot of size 250 and color red
#     #images_vid.append([ax.imshow(image,cmap='gray')])
#     images_vid.append([ax.imshow(image, cmap='gray')])
#     images_as_array.append(image)
# # The animation can now be created using ArtistAnimation
# anim = animation.ArtistAnimation(fig,
#                                 images_vid,
#                                 interval= 1000 / 30,
#                                 blit=True,
#                                 repeat=False
#                                 )
# HTML(anim.to_jshtml())

In [3]:
# import numpy as np
# # Step 1: Data Preparation
# # [index][XYZ]
# coordinates = np.load(dataPath + mouseId + "/meta/neurons/cell_motor_coordinates.npy")
# print(coordinates.shape)

# # responses = []
# # responsesPath = dataPath + mousePath + "/data/responses/"
# # for fileName in os.listdir(responsesPath):
# #     responses.append(np.load(responsesPath + fileName))

# # [index][time]
# responses = np.load(dataPath + mouseId + "/data/responses/0.npy")
# print(responses.shape)

# # [height][width][time]
# videos = np.load(dataPath + mouseId + "/data/videos/0.npy")
# print(videos.shape)

# neuronCount = responses.shape[0]
# frameCount = videos.shape[2]

# responses[0][50:]

In [6]:
# import scipy.sparse as sp
# from scipy import ndimage

# # Example sparse 3D data (COO format)
# data = [1, 2.3, 3]
# indices = [[0, 1, 2], [1, 5, 0], [2, 0, 1]]
# sparse_3d_data = sp.coo_matrix((data, (indices[0], indices[1])))
# print(sparse_3d_data)
# sparse_3d_data.shape

# zoom = (2, 0.5)
# new_sparse_3d_data = ndimage.zoom(sparse_3d_data.toarray(), zoom)  # Using nearest-neighbor interpolation
# print(new_sparse_3d_data)
# new_sparse_3d_data.shape

In [3]:
# (_, I, J, K) = a.shape
# count = 0
# indices = [(i, j, k) for i in range(I) for j in range(J) for k in range(K)]
# for i, j, k in indices:
#     if a[0][i][j][k] > 0:
#         count += 1
# count

5873

In [1]:
import numpy as np

import os

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

TRAIN_COUNT = 60
VALIDATION_COUNT = 20
TEST_COUNT = 20

Using cuda device


In [3]:
from NeurosismDataset3d import NeurosismDataset3d

dataPath = "../sensorium_data/"
mouseId = "dynamic29156-11-10-Video-8744edeac3b4d1ce16b680916b5267ce"

frameCount = 20
myDataset = NeurosismDataset3d(device, frameCount, dataPath + mouseId, densify=False, pool=1, seed=42)
a, b = myDataset[0]

batchSize = 1
shuffle = True
numWorkers = 4
neurosismDataloader = DataLoader(myDataset, batch_size=batchSize, shuffle=shuffle, num_workers=numWorkers)

a.shape, b.shape

((1, 1059, 708, 426), (1, 36, 64))

In [12]:
foo = torch.randn(1, 5, 7, 11)
bar = nn.Conv3d(1, 16, 3)(foo)
foo.shape, bar.shape

(torch.Size([1, 1, 5, 7, 11]), torch.Size([1, 16, 3, 5, 9]))

In [7]:
# Step 2: Model Definition
# from Conv4d_PyTorch.Conv4d import Conv4d

class UNet(nn.Module):
    def __init__(self, inputChannelCount: int, outputChannelCount: int):
        super(UNet, self).__init__()

        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv3d(inputChannelCount, 64, kernel_size=3, padding=1),
            nn.BatchNorm3d(64),
            nn.ReLU(inplace=True),
            # nn.MaxPool3d(kernel_size=2, stride=2),
            nn.MaxPool3d(2)
            nn.Conv3d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm3d(64),
            nn.ReLU(inplace=True),
        )

        # Decoder
        self.decoder = nn.Sequential(
            nn.Conv3d(128, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            # nn.Conv3d(64, outputChannelCount, kernel_size=3, padding=1),
            # nn.ReLU(inplace=True),
            nn.ConvTranspose3d(inputChannelCount, inputChannelCount // 2, kernel_size=2, stride=2),
        )

        # Final output layer
        self.out = nn.Conv3d(64, outputChannelCount, kernel_size=1)

    def forward(self, x):
        # Encoder
        print("1.1")
        x1 = self.encoder(x)
        # Decoder
        print("1.2")
        x = self.decoder(x1)
        # Concatenate skip connection from encoder
        print("1.3")
        x = torch.cat([x, x1], dim=1)
        # Output layer
        print("1.4")
        x = self.out(x)
        print("1.5")
        return x

model = UNet(1, 1).to(device)

In [9]:
# Step 3: Loss Function
criterion = nn.MSELoss()
# criterion = nn.CrossEntropyLoss()
# criterion = nn.BCEWithLogitsLoss()

In [10]:
# Step 4: Optimizer
# optimizer = optim.SGD(model.parameters(), lr=0.01)
# optimizer = optim.RMSprop(model.parameters(), lr=0.01)
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [11]:
# Step 5: Training Loop
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in neurosismDataloader:
        print("1")
        outputs = model(inputs)
        print("2")
        loss = criterion(outputs, labels)
        print("3")

        # backpropagation
        loss.backward()
        print("4")
        optimizer.step()
        print("5")
        optimizer.zero_grad()
        print("6")

        running_loss += loss.item()
        print("7")

    # Print average loss per epoch
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

1
torch.Size([1, 1, 100, 10948])
torch.Size([1, 32, 100, 10948])
torch.Size([1, 64, 50, 5474])
torch.Size([1, 128, 25, 2737])
torch.Size([1, 256, 12, 1368])
torch.Size([1, 512, 6, 684])
torch.Size([1, 256, 12, 1368])
torch.Size([1, 128, 25, 2737])
torch.Size([1, 64, 50, 5474])
torch.Size([1, 32, 100, 10948])
torch.Size([1, 1, 100, 10948])
2


D:\Users\divad\miniconda3\envs\py38\lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([1, 324, 36, 64])) that is different to the input size (torch.Size([1, 1, 100, 10948])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: The size of tensor a (10948) must match the size of tensor b (64) at non-singleton dimension 3

In [24]:
outputs.shape, labels.shape

(torch.Size([1, 1, 100, 10948]), torch.Size([1, 324, 36, 64]))

In [ ]:
# Step 6: Validation/Test
model.eval()
with torch.no_grad():
    for inputs, labels in val_loader:  # Assuming val_loader is DataLoader
        outputs = model(inputs)
        # Evaluate model performance (e.g., compute accuracy, loss, etc.)